In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
from sklearn.model_selection import train_test_split

filename = "encoded_data_04_13.csv"
df = pd.read_csv(f"./DataSets/EncodedData/{filename}", sep=",", encoding="UTF-8")
df.head()

,name,height_in_cm,year,goals_for,goals_against,goals,assists,red_cards,yellow_cards,minutes_played,market_value_in_eur,age_at_evaluation,log_market_value_base10,country_of_citizenship_encoded,sub_position_encoded,club_id_encoded,domestic_competition_id_encoded
0,Miroslav Klose,184.0,2012.75,31.0,14.0,11.0,1.0,0.0,6.0,1483.0,4000000.0,35,6.60206,7.191504e+06,5.331868e+06,7.836111e+06,7.182738e+06
1,Miroslav Klose,184.0,2013.25,24.0,15.0,5.0,2.0,0.0,2.0,1102.0,2000000.0,35,6.30103,7.191504e+06,5.331868e+06,7.836111e+06,7.182738e+06
2,Miroslav Klose,184.0,2013.75,14.0,21.0,4.0,2.0,0.0,1.0,950.0,1000000.0,36,6.00000,7.191504e+06,5.331868e+06,7.836111e+06,7.182738e+06
3,Miroslav Klose,184.0,2014.25,24.0,22.0,4.0,3.0,0.0,1.0,1270.0,1000000.0,36,6.00000,7.191504e+06,5.331868e+06,7.836111e+06,7.182738e+06
4,Miroslav Klose,184.0,2014.75,31.0,18.0,4.0,3.0,0.0,2.0,496.0,1000000.0,37,6.00000,7.191504e+06,5.331868e+06,7.836111e+06,7.182738e+06


In [4]:
df = df.sample(n=5000, random_state=42)

In [6]:
# log_market_value_base10
df = df.drop(columns=['name', 'market_value_in_eur'])
# Del dataene inn i treningssett og testsett
train_data, test_data = train_test_split(df, test_size=0.25, random_state=42)  # Juster test_size og random_state etter behov

# Konverter dataene til TabularDataset-objekter
train_data = TabularDataset(train_data)
test_data = TabularDataset(test_data)
train_data.head()



,height_in_cm,year,goals_for,goals_against,goals,assists,red_cards,yellow_cards,minutes_played,age_at_evaluation,log_market_value_base10,country_of_citizenship_encoded,sub_position_encoded,club_id_encoded,domestic_competition_id_encoded
120238,177.0,2023.25,10.0,7.0,0.0,1.0,0.0,1.0,125.0,19,6.602060,3.902287e+06,5.865733e+06,7.953486e+06,2.483243e+06
110501,180.0,2020.75,20.0,15.0,0.0,1.0,0.0,1.0,580.0,19,5.397940,1.912235e+06,3.362303e+06,3.677499e+05,6.786176e+05
77896,179.0,2015.75,3.0,1.0,0.0,0.0,0.0,0.0,120.0,22,7.113943,3.902287e+06,3.650476e+06,3.530455e+06,8.084417e+06
15492,188.0,2016.75,8.0,3.0,0.0,0.0,0.0,0.0,450.0,31,7.176091,8.564591e+06,4.318791e+06,6.743160e+06,2.390848e+06
66588,190.0,2020.25,1.0,12.0,0.0,0.0,0.0,0.0,519.0,29,5.511883,3.559251e+06,5.331868e+06,4.500000e+05,2.118387e+06


In [7]:
from autogluon.tabular import TabularPredictor

# Create and train the model with a specified time limit
predictor = TabularPredictor(label='log_market_value_base10', problem_type="regression", eval_metric="mean_absolute_percentage_error").fit(train_data, presets='high_quality', time_limit=300)  # This will limit the training to one hour (3600 seconds)

# Evaluate the model on the test set
performance = predictor.evaluate(test_data)
print(performance)


No path specified. Models will be saved in: "AutogluonModels\ag-20240414_104707"
Presets specified: ['high_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 300 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels\ag-20240414_104707/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 75s
AutoGluon will save models to "AutogluonModels\ag-20240414_104707/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  1.0.

[1000]	valid_set's l2: 0.0914837	valid_set's mean_absolute_percentage_error: -0.0594576


	-0.0358	 = Validation score   (-mean_absolute_percentage_error)
	9.2s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 40.6s of the 65.59s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0361	 = Validation score   (-mean_absolute_percentage_error)
	6.53s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 34.01s of the 59.0s of remaining time.
	-0.038	 = Validation score   (-mean_absolute_percentage_error)
	0.71s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 33.09s of the 58.08s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0354	 = Validation score   (-mean_absolute_percentage_error)
	11.78s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesMSE

[1000]	valid_set's l2: 0.0837082	valid_set's mean_absolute_percentage_error: -0.0534484
[1000]	valid_set's l2: 0.0784802	valid_set's mean_absolute_percentage_error: -0.0550563
[1000]	valid_set's l2: 0.0836735	valid_set's mean_absolute_percentage_error: -0.0525123


	-0.0359	 = Validation score   (-mean_absolute_percentage_error)
	9.48s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 136.94s of the 210.28s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-0.036	 = Validation score   (-mean_absolute_percentage_error)
	6.6s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 130.28s of the 203.62s of remaining time.
	-0.0379	 = Validation score   (-mean_absolute_percentage_error)
	0.77s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 129.3s of the 202.64s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0352	 = Validation score   (-mean_absolute_percentage_error)
	12.46s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraT

...

[1000]	valid_set's l2: 0.0731101	valid_set's mean_absolute_percentage_error: -0.0535388
[1000]	valid_set's l2: 0.088671	valid_set's mean_absolute_percentage_error: -0.0561145
[2000]	valid_set's l2: 0.0827618	valid_set's mean_absolute_percentage_error: -0.0545159


	Ran out of time, early stopping on iteration 2722. Best iteration is:
	[2599]	valid_set's l2: 0.0812279	valid_set's mean_absolute_percentage_error: -0.0540141


[1000]	valid_set's l2: 0.0844084	valid_set's mean_absolute_percentage_error: -0.0573385
[1000]	valid_set's l2: 0.0823023	valid_set's mean_absolute_percentage_error: -0.0543443


	-0.0369	 = Validation score   (-mean_absolute_percentage_error)
	16.45s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetTorch_r22_BAG_L1 ... Training model for up to 16.3s of the 89.64s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
		Unable to import dependency torch
A quick tip is to install via `pip install torch`.
The minimum torch version is currently 2.0.
Fitting model: XGBoost_r33_BAG_L1 ... Training model for up to 16.18s of the 89.52s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0379	 = Validation score   (-mean_absolute_percentage_error)
	15.59s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: ExtraTrees_r42_BAG_L1 ... Training model for up to 0.43s of the 73.77s of remaining time.
	-0.0383	 = Validation score   (-mean_absolute_percentage_error)
	0.58s	 = Training   runtime
	0.18s	 = Validation runtime
Fitti

{'mean_absolute_percentage_error': -0.033569599113308764, 'root_mean_squared_error': -0.26668074499419625, 'mean_squared_error': -0.07111861975065954, 'mean_absolute_error': -0.20469663255143136, 'r2': 0.8318228682940496, 'pearsonr': 0.9126174804350475, 'median_absolute_error': -0.15983944750357182}


In [17]:
# Get the MAPEs of all models
model_scores = predictor.get_model_best()
    
# Sort the models by MAPE in ascending order and get the top 10 models
top_10_models = sorted(model_scores.items(), key=lambda x: x[1])[:10]

# Print the MAPEs of the top 10 models
print("Top 10 Models by MAPE:")
for model, mape in top_10_models:
    print(f"{model}: {mape}")


C:\Users\mg\AppData\Local\Temp\ipykernel_20888\4055244273.py:2: DeprecationWarning: `get_model_best` has been deprecated and will be removed in version 1.2. Please use `model_best` instead. This will raise an error in the future!
  model_scores = predictor.get_model_best()


AttributeError: 'str' object has no attribute 'items'

In [8]:
y_pred = predictor.predict(test_data.drop(columns=["log_market_value_base10"]))
y_pred.head()

67877     5.528651
118623    5.095131
70125     5.823779
15245     5.565951
75534     5.837325
Name: log_market_value_base10, dtype: float32

In [9]:
predictor.evaluate(test_data, silent=True)

{'mean_absolute_percentage_error': -0.033569599113308764,
 'root_mean_squared_error': -0.26668074499419625,
 'mean_squared_error': -0.07111861975065954,
 'mean_absolute_error': -0.20469663255143136,
 'r2': 0.8318228682940496,
 'pearsonr': 0.9126174804350475,
 'median_absolute_error': -0.15983944750357182}

In [10]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3_FULL,-0.033570,NaN,mean_absolute_percentage_error,0.576031,NaN,10.661706,0.001993,NaN,0.505998,3,True,54
1,WeightedEnsemble_L2_FULL,-0.033604,NaN,mean_absolute_percentage_error,0.043510,NaN,2.863521,0.002079,NaN,0.396521,2,True,43
2,LightGBMXT_BAG_L1_FULL,-0.033661,NaN,mean_absolute_percentage_error,0.009965,NaN,0.242708,0.009965,NaN,0.242708,1,True,30
3,CatBoost_BAG_L2_FULL,-0.033665,NaN,mean_absolute_percentage_error,0.563068,NaN,10.156635,0.002995,NaN,0.440214,2,True,47
4,LightGBMXT_BAG_L2_FULL,-0.033679,NaN,mean_absolute_percentage_error,0.565555,NaN,9.860418,0.005482,NaN,0.143997,2,True,44
5,CatBoost_r177_BAG_L2_FULL,-0.033735,NaN,mean_absolute_percentage_error,0.565095,NaN,10.003405,0.005022,NaN,0.286984,2,True,51
6,CatBoost_r177_BAG_L1_FULL,-0.033757,NaN,mean_absolute_percentage_error,0.004559,NaN,0.729872,0.004559,NaN,0.729872,1,True,37
7,ExtraTreesMSE_BAG_L2_FULL,-0.033778,NaN,mean_absolute_percentage_error,0.665226,NaN,10.339366,0.105153,0.156708,0.622946,2,True,48
8,CatBoost_BAG_L1_FULL,-0.034026,NaN,mean_absolute_percentage_error,0.002986,NaN,0.911979,0.002986,NaN,0.911979,1,True,33
9,CatBoost_r9_BAG_L1_FULL,-0.034034,NaN,mean_absolute_percentage_error,0.013588,NaN,3.482768,0.013588,NaN,3.482768,1,True,39


In [11]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                            model  score_val                     eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0             WeightedEnsemble_L3  -0.036882  mean_absolute_percentage_error       1.479882  155.058021                0.000000           0.500474            3      False         30
1             WeightedEnsemble_L2  -0.036913  mean_absolute_percentage_error       0.315790   66.444216                0.000000           0.457909            2      False         19
2               LightGBMXT_BAG_L1  -0.037205  mean_absolute_percentage_error       0.042042    6.106277                0.042042           6.106277            1      False          3
3              CatBoost_r9_BAG_L1  -0.037294  mean_absolute_percentage_error       0.042594   33.278464                0.042594          33.278464            1      False         12
4            CatBoost_r177_B

c:\Users\mg\Documents\GitHub\MachineLearning\.venv\Lib\site-packages\autogluon\core\utils\plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


In [12]:
leaderboard = predictor.leaderboard(test_data)
leaderboard


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L1_FULL,-0.036051,NaN,mean_absolute_percentage_error,0.008775,NaN,0.233830,0.008775,NaN,0.233830,1,True,33
1,WeightedEnsemble_L2_FULL,-0.036085,NaN,mean_absolute_percentage_error,0.144237,NaN,5.285860,0.002003,NaN,0.457909,2,True,49
2,WeightedEnsemble_L3_FULL,-0.036129,NaN,mean_absolute_percentage_error,1.069866,NaN,14.774684,0.004059,NaN,0.500474,3,True,60
3,CatBoost_r137_BAG_L1_FULL,-0.036230,NaN,mean_absolute_percentage_error,0.002990,NaN,0.637915,0.002990,NaN,0.637915,1,True,46
4,CatBoost_BAG_L1_FULL,-0.036292,NaN,mean_absolute_percentage_error,0.003498,NaN,0.696590,0.003498,NaN,0.696590,1,True,36
5,CatBoost_r177_BAG_L2_FULL,-0.036339,NaN,mean_absolute_percentage_error,0.768916,NaN,10.874545,0.004493,NaN,0.242666,2,True,57
6,LightGBM_r96_BAG_L1_FULL,-0.036394,NaN,mean_absolute_percentage_error,0.017589,NaN,0.359886,0.017589,NaN,0.359886,1,True,43
7,CatBoost_r9_BAG_L1_FULL,-0.036420,NaN,mean_absolute_percentage_error,0.010964,NaN,2.423258,0.010964,NaN,2.423258,1,True,42
8,CatBoost_BAG_L2_FULL,-0.036446,NaN,mean_absolute_percentage_error,0.769367,NaN,11.049819,0.004944,NaN,0.417941,2,True,53
9,CatBoost_r9_BAG_L2_FULL,-0.036457,NaN,mean_absolute_percentage_error,0.771400,NaN,12.156210,0.006977,NaN,1.524332,2,True,59
